In [ ]:
using PyCall
using RCall
using DecisionTree
using DataFrames
using ScikitLearn

In [ ]:
R"mydata=iris"

In [ ]:
@rget mydata

In [ ]:
x=10

In [ ]:
@rput x

In [ ]:
R"x+1"

In [ ]:
first(mydata,5)

In [ ]:
using Gadfly

In [ ]:
iris = mydata
Gadfly.plot(iris,x=:Sepal_Length,y=:Sepal_Width,color=:Species)

In [ ]:
R"""
library(ggplot2)

ggplot(iris,aes(x=Sepal.Length,y=Sepal.Width))+geom_point(aes(color=Species))
"""

In [ ]:
py"""
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA

# import some data to play with
iris = datasets.load_iris()
X = iris.data[:, :2]  # we only take the first two features.
y = iris.target

x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

plt.figure(2, figsize=(8, 6))
plt.clf()

# Plot the training points
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Set1,
            edgecolor='k')
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())

# To getter a better understanding of interaction of the dimensions
# plot the first three PCA dimensions
fig = plt.figure(1, figsize=(8, 6))
ax = Axes3D(fig, elev=-150, azim=110)
X_reduced = PCA(n_components=3).fit_transform(iris.data)
ax.scatter(X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y,
           cmap=plt.cm.Set1, edgecolor='k', s=40)
ax.set_title("First three PCA directions")
ax.set_xlabel("1st eigenvector")
ax.w_xaxis.set_ticklabels([])
ax.set_ylabel("2nd eigenvector")
ax.w_yaxis.set_ticklabels([])
ax.set_zlabel("3rd eigenvector")
ax.w_zaxis.set_ticklabels([])

plt.show()
"""

### Let's visualize the data

In [ ]:
using DataVoyager
using VegaDatasets

In [ ]:
mydata |> Voyager()

In [ ]:
using VegaDatasets
cars=dataset("cars")
cars |> Voyager()

In [ ]:
iris = mydata

###   Scikitlearn classifiers using Julia bindings of Python (PyCall.jl)

In [ ]:
using PyCall
LNM = pyimport("sklearn.linear_model");
learner=LNM.LogisticRegression()

x=iris[:,1:4] |> Matrix;
y=iris[:,5] |> Array;

model=learner.fit(x, y)

accuracy = sum(model.predict(x) .== y) / length(y)
println("accuracy: $accuracy")

In [ ]:
learner

In [ ]:
#@sk_import neighbors: KNeighborsClassifier
KNN = pyimport("sklearn.neighbors")

using ScikitLearn.CrossValidation: cross_val_score

modelKNN = KNN.KNeighborsClassifier(3)
cross_val_score(modelKNN, x, y; cv=10)  # 5-fold

In [ ]:
using ScikitLearn.GridSearch: GridSearchCV
GS=pyimport("sklearn.model_selection");
lbgfs=LNM.LogisticRegression(multi_class="auto",solver="lbfgs")

In [ ]:
gridsearch = GS.GridSearchCV(lbgfs, Dict(:C => 0.1:0.1:2.0))
res=gridsearch.fit(x, y)
println("Best parameters: $(res.best_params_)")

In [ ]:
using Plots
gr()
Cparams=convert.(Float64,res.cv_results_["param_C"])
Plots.plot(Cparams,res.cv_results_["std_test_score"])

In [ ]:
using PyCall
const ENS=pyimport("sklearn.ensemble")
const LM=pyimport("sklearn.linear_model")
const DA=pyimport("sklearn.discriminant_analysis")
const NN=pyimport("sklearn.neighbors")
const SVM=pyimport("sklearn.svm")
const TREE=pyimport("sklearn.tree")
const ANN=pyimport("sklearn.neural_network")
const GP=pyimport("sklearn.gaussian_process")
const KR=pyimport("sklearn.kernel_ridge")
const NB=pyimport("sklearn.naive_bayes")
const ISO=pyimport("sklearn.isotonic")

In [ ]:
learners = Dict(
  "AdaBoostClassifier" => ENS.AdaBoostClassifier,
  "BaggingClassifier" => ENS.BaggingClassifier,
  "ExtraTreesClassifier" => ENS.ExtraTreesClassifier,
  "VotingClassifier" => ENS.VotingClassifier,
  "GradientBoostingClassifier" => ENS.GradientBoostingClassifier,
  "RandomForestClassifier" => ENS.RandomForestClassifier,
  "LDA" => DA.LinearDiscriminantAnalysis,
  "QDA" => DA.QuadraticDiscriminantAnalysis,
  "LogisticRegression" => LM.LogisticRegression,
  "PassiveAggressiveClassifier" => LM.PassiveAggressiveClassifier,
  "RidgeClassifier" => LM.RidgeClassifier,
  "RidgeClassifierCV" => LM.RidgeClassifierCV,
  "SGDClassifier" => LM.SGDClassifier,
  "KNeighborsClassifier" => NN.KNeighborsClassifier,
  "RadiusNeighborsClassifier" => NN.RadiusNeighborsClassifier,
  "NearestCentroid" => NN.NearestCentroid,
  "SVC" => SVM.SVC,
  "LinearSVC" => SVM.LinearSVC,
  "NuSVC" => SVM.NuSVC,
  "MLPClassifier" => ANN.MLPClassifier,
  "GaussianProcessClassifier" => GP.GaussianProcessClassifier,
  "DecisionTreeClassifier" => TREE.DecisionTreeClassifier,
  "GaussianNB" => NB.GaussianNB,
  "MultinomialNB" => NB.MultinomialNB,
  "ComplementNB" => NB.ComplementNB,
  "BernoulliNB" => NB.BernoulliNB
);

In [ ]:
function sktrain(learner,x,y)
    model = learner()
    xx = x |> Matrix
    yy = y |> Array
    model.fit(xx,yy)
    return model
end

In [ ]:
function skpredict(themodel,x)
    xx = x |> Matrix
    return themodel.predict(xx) |> collect
end

In [ ]:
x=iris[:,1:4] |> Matrix;
y=iris[:,5] |> Array;

In [ ]:
modelLR = sktrain(learners["LogisticRegression"],x,y)

In [ ]:
res = skpredict(modelLR,x)

In [ ]:
sum(res .== y) / length(y)

### Let's use R's Caret to predict Iris species

In [ ]:
using RCall
R"
library(ggplot2)
ggplot(iris,aes(Sepal.Length,Petal.Width,color=Species))+geom_point()
"

In [ ]:
x=rand(10)
R"t.test($x)"

In [ ]:
R"val=runif(10)"
@rget(val)

In [ ]:
x=sin.(-2π:0.1:2π)
@rput(x)
R"plot(x,type='l')";

In [ ]:
using RCall
R"""
library(caret)
library(e1071)
library(randomForest)
library(xgboost)
"""

In [ ]:
function crtrain(learner,x::DataFrame,y::Vector)
    xx = x |> DataFrame # DataFrame(x)
    yy = y |> Vector
    model = rcall(:train,xx,yy,method=learner)
    return model
end

In [ ]:
crt

In [ ]:
function crtpredict(model,x)
    xx = x |> DataFrame
    res = rcall(:predict,model,xx) #in robj
    return rcopy(res) |> collect  # return extracted robj
end

In [ ]:
x

In [ ]:
crtmodel = crtrain("rf",x,y)

In [ ]:
methods(crtrain)

In [ ]:
res = crtpredict(crtmodel,x) 

In [ ]:
sum(res .== y) / length(y)

In [ ]:
crtmodel = crtrain("xgbTree",x,y)
res = crtpredict(crtmodel,x) 
sum(res .== y) / length(y)